In [1]:
# get code from github
!git clone https://github.com/Wensi-Tang/OS-CNN.git
#change path to OS-CNN
%cd /content/OS-CNN

Cloning into 'OS-CNN'...
remote: Enumerating objects: 557, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 557 (delta 1), reused 4 (delta 0), pack-reused 550
Receiving objects: 100% (557/557), 49.18 MiB | 26.01 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/OS-CNN


In [8]:
import torch
from os.path import dirname
from utils.dataloader.TSC_data_loader import TSC_data_loader
from Classifiers.OS_CNN.OS_CNN_easy_use import OS_CNN_easy_use
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from tqdm import tqdm

In [9]:
cuda_available = torch.cuda.is_available()
print("Are we using GPU?: ", cuda_available)
if(cuda_available):
    print("the GPU name is: ",torch.cuda.get_device_name(0))

Are we using GPU?:  True
the GPU name is:  Tesla T4


In [10]:
btc_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1-FmXzXANW85DmNbKKKoDkZr7w3cZEZi1', parse_dates=['open_time'])
btc_df = btc_df.sort_values('open_time').reset_index(drop=True)
btc_df = btc_df[['open', 'high', 'low', 'close', 'volume']]
btc_df['label'] = (btc_df['open'] < btc_df['close']).astype(int)
btc_df.head()

,open,high,low,close,volume,label
0,36060.5,36060.5,35807.9,35815.6,10.423,0
1,35815.5,35939.5,35717.3,35722.0,9.339,0
2,35715.6,35849.5,35715.6,35723.4,3.884,1
3,35735.1,35735.1,35720.0,35727.6,1.684,0
4,35727.2,35850.0,35714.0,35831.6,3.474,1


In [11]:
train_df = btc_df.iloc[:int(len(btc_df)*0.8),:]
test_df = btc_df.iloc[int(len(btc_df)*0.8):,:]

In [12]:
def create_sequences(input_data: pd.DataFrame, target_column, sequence_length):
    sequences = []
    data_size = len(input_data)
    
    for i in tqdm(range(data_size - sequence_length)):
        seq = input_data[i:i+sequence_length]
        label = input_data.iloc[i + sequence_length][target_column]
        sequences.append((seq, label))
    return sequences

In [13]:
SEQUENCE_LENGTH = 30
train_sequences = create_sequences(train_df, 'label', SEQUENCE_LENGTH)
test_sequences = create_sequences(test_df, 'label', SEQUENCE_LENGTH)
train_sequences[0][0].dtypes, type(train_sequences[0][1])

100%|██████████| 25528/25528 [00:03<00:00, 6618.72it/s]


(open      float64
 high      float64
 low       float64
 close     float64
 volume    float64
 label       int64
 dtype: object, numpy.float64)

In [14]:
X_train = np.float32(train_df.iloc[:,:-2].values)
y_train = np.int64(train_df.iloc[:,-1].values)

X_test = np.float32(test_df.iloc[:,:-2].values)
y_test = np.int64(test_df.iloc[:,-1].values)

print(f'train length: {len(X_train)}')
print(f'test length: {len(X_test)}')
print('unique train label',np.unique(y_train))
print('unique test label',np.unique(y_test))

train length: 102230
test length: 25558
unique train label [0 1]
unique test label [0 1]


In [15]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((102230, 4), (102230,), (25558, 4), (25558,))

In [16]:
Result_log_folder = './Results_of_OS_CNN_for_multivariate/OS_CNN_result_iter_0/'
dataset_name = "btc"

In [23]:
# creat model and log save place
model = OS_CNN_easy_use(
    Result_log_folder = Result_log_folder, # the Result_log_folder
    dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder
    device = "cuda:0",                     # Gpu 
    max_epoch = 500,                        # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough
    paramenter_number_of_layer_list = [8*128*X_train.shape[1], 5*128*256 + 2*256*128]
    )


In [24]:
model.fit(X_train, y_train, X_test, y_test)

code is running on  cuda:0
epoch = 49 lr =  0.001
train_acc=	 0.6485865205908247 	 test_acc=	 0.5694107520150247 	 loss=	 0.043732330203056335
log saved at:
./Results_of_OS_CNN/OS_CNN_result_iter_0/btc/btc_.txt
epoch = 99 lr =  0.001
train_acc=	 0.7045094394991686 	 test_acc=	 0.6399561781047031 	 loss=	 0.6447405219078064
log saved at:
./Results_of_OS_CNN/OS_CNN_result_iter_0/btc/btc_.txt
epoch = 149 lr =  0.0005
train_acc=	 0.6526655580553653 	 test_acc=	 0.5465998904452618 	 loss=	 0.14860044419765472
log saved at:
./Results_of_OS_CNN/OS_CNN_result_iter_0/btc/btc_.txt
epoch = 199 lr =  0.00025
train_acc=	 0.7998434901692263 	 test_acc=	 0.6202363252210659 	 loss=	 0.6713371872901917
log saved at:
./Results_of_OS_CNN/OS_CNN_result_iter_0/btc/btc_.txt
epoch = 249 lr =  0.000125
train_acc=	 0.7032573608529786 	 test_acc=	 0.576570936693012 	 loss=	 0.009805042296648026
log saved at:
./Results_of_OS_CNN/OS_CNN_result_iter_0/btc/btc_.txt
epoch = 299 lr =  0.0001
train_acc=	 0.71903550816

In [25]:
y_predict = model.predict(X_test)

print('correct:',y_test)
print('predict:',y_predict)

correct: [0 1 0 ... 0 1 0]
predict: [0. 1. 0. ... 0. 1. 0.]


In [26]:
acc = accuracy_score(y_predict, y_test)
print(acc)

0.6682447765865873
